# Job Word Embeddings

here are some goals that I want to accomplish in this notebook
 - clean our granted patent dataset to only have utility work, and have only patent abstract, patent date, and patent id columns (in case we want to use g_cpc_current)
 - clean our kaggle dataset by removing stop words to get relevant words. Then, tokenize, and get word embeddings using gensim
 - clean our g_patent similarily
 - start to use bag of words and tf-idf to get similarities between the job postings and g_patents in nearby years (+/- 2 or 3 years)

## Imports and Reading in Files

In [1]:
# Here, we are going to load the generic g_patent and g_cpc_current DataFrames as well as our pandas, matplotlib etc
import pandas as pd
import gensim
import nltk
from nltk.corpus import stopwords 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
sns.set( style = 'white' )

# Here is a variable change in the width of the string for the columns. I am placing it here as it may need
# to be changed often, so I will move it right here!
pd.options.display.max_colwidth = 1000

# this is the english stop words list
eng_stp_wrds = stopwords.words('english')

In [2]:
# here is the granted patent dataset that we will use alongside the job posting abstracts
df_patent = pd.read_csv("../g_patent.tsv", delimiter='\t', dtype={'patent_id': str, 
                                                        'patent_type': str, 
                                                        'patent_title': str,
                                                        'patent_abstract': str,
                                                        'wipo_kind': str,
                                                        'num_claims': int,
                                                        'withdrawn': int,
                                                        'filename': str}, parse_dates=[2])
df_patent.drop(df_patent[df_patent['patent_type'] != 'utility'].index, inplace=True)
df_patent.drop(axis=1, columns=["patent_type", "patent_title", "wipo_kind","num_claims", "withdrawn", "filename"], inplace=True)
df_patent.head(20) 

,patent_id,patent_date,patent_abstract
0,10000000,2018-06-19,"A frequency modulated (coherent) laser detection and ranging system includes a read-out integrated circuit formed with a two-dimensional array of detector elements each including a photosensitive region receiving both return light reflected from a target and light from a local oscillator, and local processing circuitry sampling the output of the photosensitive region four times during each sample period clock cycle to obtain quadrature components. A data bus coupled to one or more outputs of each of the detector elements receives the quadrature components from each of the detector elements for each sample period and serializes the received quadrature components. A processor coupled to the data bus receives the serialized quadrature components and determines an amplitude and a phase for at least one interfering frequency corresponding to interference between the return light and the local oscillator light using the quadrature components."
1,10000001,2018-06-19,"The injection molding machine includes a fixed platen, a moveable platen moving forward and backward by a toggle link, a base plate supporting the toggle link, a driving part for mold clamping to operate the toggle link, a driving part for mold thickness adjustment to adjust a mold thickness, and a control unit to calculate a movement distance gap before a clamping process by controlling the driving part for mold thickness adjustment to move the base plate backward and then move the base plate forward to a target movement position based on a fold amount of the toggle link, and control the driving part for mold thickness adjustment using a value obtained by deducting the movement distance gap from the fold amount of the toggle link when producing a clamp force."
2,10000002,2018-06-19,"The present invention relates to: a method for manufacturing a polymer film, the method including a base film forming step for co-extruding a first resin containing a polyamide-based resin and a second resin containing a copolymer including polyamide-based segments and polyether-based segments; a co-extruded film including a base film including a first resin layer containing a polyamide-based resin, and a second resin layer containing a copolymer having polyamide-based segments and polyether-based segments; to a co-extruded film including a base film including a first resin layer and a second resin layer, which have different melting points; and to a method for manufacturing a polymer film, the method including a base film forming step including a step of co-extruding a first resin and a second resin, which have different melting points."
3,10000003,2018-06-19,"The invention relates to a method for producing a container (2) from a thermoplastic, having at least one surround (4), provided in the container wall (1), for a container opening. The surround (4) comprises a structure behind which parts of the container wall (1) extend and/or which is penetrated by said parts. The method is carried out using a multi-part blow mold that has at least two mold parts, each having at least one cavity, wherein the surround is placed as an insert in the cavity (10) of the blow mold (7). The method comprises pressing the preform that has been forced into the cavity (10) into the structure of the surround (4) by means of a tool which is brought to bear on the preform (12) on the side of the preform facing away from the cavity (10)."
4,10000004,2018-06-19,"The present invention relates to provides a double-oriented film, co-extrude, and of low thickness, with a layered composition that gives the property of being of high barrier to gases and manufactured by the process of co-extrusion of 3 bubbles, which gives the property of when being thermoformed, ensure the distribution of uniform thickness in the walls, base, folds, and corners of the formed tray saving a minimum of 50% of plastic without diminishing its gas barrier and its resistance to punct

In [3]:
df_patent['patent_abstract'].fillna("filled spot for NaN", inplace=True) # there was apparently some NaN spots in the abstract column

In [4]:
df_cpc = pd.read_table("../g_cpc_current.tsv", delimiter="\t", dtype={"patent_id": str,
                                                               "cpc_sequence": int,
                                                               "cpc_section": str,
                                                               "cpc_subclass": str,
                                                               "cpc_group": str,
                                                               "cpc_type": str,
                                                               "cpc_symbol_position": str})
df_cpc.head(20) # this may not be used within this specific notebook, but it is here loaded in case we want to check the type of patents

,patent_id,cpc_sequence,cpc_section,cpc_class,cpc_subclass,cpc_group,cpc_type
0,5664589,0,A,A45,A45D,A45D2/20,inventional
1,10439720,3,G,G02,G02B,G02B6/3652,inventional
2,9840937,2,F,F01,F01D,F01D25/30,inventional
3,7407213,1,B,B62,B62D,B62D33/03,inventional
4,11014297,4,B,B29,B29C,B29C64/245,inventional
5,6959012,0,H,H04,H04J,H04J3/0685,inventional
6,6725745,0,H,H01,H01H,H01H85/0208,inventional
7,8625669,7,H,H04,H04N,H04N19/13,inventional
8,11011577,16,H,H10,H10B,H10B63/20,inventional
9,5087721,1,A,A61,A61K,A61K31/685,inventional


In [5]:
online_df = pd.read_csv('../onlinejobpostings.csv') # here we have the online job posting dataset that was on kaggle
online_df['date'] = pd.to_datetime(online_df['date'], errors = 'coerce')
online_df.drop(axis=1, inplace=True, columns=online_df.columns[4:])
online_df

,jobpost,date,Title,Company
0,"AMERIA Investment Consulting Company\r\nJOB TITLE: Chief Financial Officer\r\nPOSITION LOCATION: Yerevan, Armenia\r\nJOB DESCRIPTION: AMERIA Investment Consulting Company is seeking a\r\nChief Financial Officer. This position manages the company's fiscal and\r\nadministrative functions, provides highly responsible and technically\r\ncomplex staff assistance to the Executive Director. The work performed\r\nrequires a high level of technical proficiency in financial management\r\nand investment management, as well as management, supervisory, and\r\nadministrative skills.\r\nJOB RESPONSIBILITIES: \r\n- Supervises financial management and administrative staff, including\r\nassigning responsibilities, reviewing employees' work processes and\r\nproducts, counseling employees, giving performance evaluations, and\r\nrecommending disciplinary action;\r\n- Serves as member of management team participating in both strategic\r\nand operational planning for the company;\r\n- Directs and ove...",2004-01-05,Chief Financial Officer,AMERIA Investment Consulting Company
1,"International Research & Exchanges Board (IREX)\r\nTITLE: Full-time Community Connections Intern (paid internship)\r\nDURATION: 3 months\r\nLOCATION: IREX Armenia Main Office; Yerevan, Armenia \r\nDESCRIPTION: IREX currently seeks to fill the position of a paid\r\nIntern for the Community Connections (CC) Program. The position is based\r\nin the Yerevan office however applicants must be willing to travel\r\nthroughout Armenia as necessary. This position reports directly to the\r\nCC Program Manager.\r\nRESPONSIBILITIES: \r\n- Presenting the CC program to interested parties; \r\n- Assisting in planning and scheduling of programmatic meetings and\r\nevents (this includes coordinating logistics for CC staff, visitors and\r\nparticipants);\r\n- Assisting the Program Staff;\r\n- Translation/Interpretation from Armenian to English and vice versa;\r\n- Helping create, maintain and update the CC filing system and\r\ndatabases;\r\n- Completing general administrative tasks for the CC...",2004-01-07,Full-time Community Connections Intern (paid internship),International Research & Exchanges Board (IREX)
2,"Caucasus Environmental NGO Network (CENN)\r\nJOB TITLE: Country Coordinator\r\nPOSITION DURATION: Renewable annual contract\r\nPOSITION LOCATION: Yerevan, Armenia\r\nJOB DESCRIPTION: Public outreach and strengthening of a growing\r\nnetwork of environmental NGOs, businesses, international organizations\r\nand public agencies. Will serve as primary contact between CENN and\r\npublic. This is a full-time position.\r\nJOB RESPONSIBILITIES: \r\n- Working with the Country Director to provide environmental information\r\nto the general public via regular electronic communications and serving\r\nas the primary local contact to Armenian NGOs and businesses and the\r\nArmenian offices of international organizations and agencies;\r\n- Helping to organize and prepare CENN seminars/ workshops;\r\n- Participating in defining the strategy and policy of CENN in Armenia,\r\nthe Caucasus region and abroad.\r\nREQUIRED QUALIFICATIONS: \r\n- Degree in environmentally related field, or 5 years ...",2004-01-07,Country Coordinator,Caucasus Environmental NGO Network (CENN)
3,"Manoff Group\r\nJOB TITLE: BCC Specialist\r\nPOSITION LOCATION: Manila, Philippines\r\nJOB DESCRIPTION: The LEAD (Local Enhancement and Development for\r\nHealth) BCC Specialist will apply state-of-the-art approaches in working\r\nwith LGUs (Local Government Units) and NGOs to help them to identify and\r\naddress provider-caused barriers to service provision as well as to\r\nidentify and address supports for good service delivery by developing\r\ntools that may be adapted to each LGU's needs. S/he will work with LEAD\r\nstaff across all components to support quality service delivery and will\r\nalso monitor implementation of improved service delivery in LGUs, and\r\nwill provide additional assistance to LG

## Word Embedding Code

In [6]:
# We are going to be using sklearn to do the cosine similarity comparisons using the tfidf method
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
vectorizer = TfidfVectorizer()

In [36]:
remote_jobs = pd.concat([online_df.loc[online_df['jobpost'].str.contains("remote", case=True)],
                        online_df.loc[online_df['jobpost'].str.contains("WFH", case=True)],
                        online_df.loc[online_df['jobpost'].str.contains("work from home", case=True)],
                        online_df.loc[online_df['jobpost'].str.contains("mobile work", case=True)],
                        online_df.loc[online_df['jobpost'].str.contains("virtual", case=True)],
                        online_df.loc[online_df['jobpost'].str.contains("online meeting", case=True)],
                        online_df.loc[online_df['jobpost'].str.contains("distributed work", case=True)]], ignore_index=True) # we are getting remote job online posts only
remote_jobs['date'].fillna("", inplace=True)
remote_jobs

,jobpost,date,Title,Company
0,"World Vision Armenia\r\nJOB TITLE: Project Assistant\r\nPOSITION LOCATION: Yerevan, Armenia\r\nJOB DESCRIPTION: World Vision Armenia announces a full-time position\r\nfor Project Assistant for the implementation of a Mobile Medical Teams\r\nand Primary Health care project. The position is based in World Vision\r\nArmenia' National office, Yerevan with extensive countrywide travel.\r\nCandidates must be flexible team players willing to travel extensively\r\nto field locations. \r\nThe Project Assistant will support the Yerevan based MMT staff with\r\nmiscellaneous administrative and project implementation duties.\r\nJOB RESPONSIBILITIES: \r\n- Provide daily administrative and technical support to the MMT Program\r\ncoordinator and Health Program Manager in implementation of the MMT\r\nProgram Activities in the sites;\r\n- Provide minor procurement, registration of drugs and other medical\r\nsupplies, customs clearance and additional support to field staff as\r\nrequired;\r\n...",2004-02-22,Project Assistant,World Vision Armenia
1,"World Vision Armenia\r\nJOB TITLE: MMT Project Manager\r\nPOSITION LOCATION: Yerevan, Armenia\r\nJOB DESCRIPTION: World Vision Armenia announces a full-time position\r\nfor MMT Project Manager for the implementation of a Mobile Medical Teams\r\nand Primary Health care project. The position is based in World Vision\r\nArmenia' National office, Yerevan with extensive countrywide travel.\r\nCandidates must be flexible team players willing to travel extensively\r\nto field locations. \r\nMMT project Manager will lead and work with other members of the MMT\r\nteam. This position is responsible for immediate Management and\r\noversight of program implementation, monitoring and evaluation,\r\nreporting.\r\nJOB RESPONSIBILITIES: As a senior member of the MMT and reporting to\r\nthe Health Program Manager, the MMT Project Manager will manage a team\r\nof three people (two health coordinators and an assistant) for the first\r\nyear of program implementation that will gradually expand...",2004-02-22,MMT Project Manager,World Vision Armenia
2,"World Vision Armenia\r\nJOB TITLE: Health Coordinators (two positions are open)\r\nJOB DESCRIPTION: World Vision Armenia announces full-time positions\r\nfor Health Coordinators for the implementation of a Mobile Medical Teams\r\nand Primary Health care project. The positions are based in World Vision\r\nArmenia' National office, Yerevan with extensive countrywide travel.\r\nCandidates must be flexible team players willing to travel extensively\r\nto field locations. \r\nMMT Health Coordinators will be responsible for direct coordination,\r\nsupervision and technical monitoring of the program success and\r\nconstrains in Lori and Gegharkunik. \r\nJOB RESPONSIBILITIES: As part of MMT team, each Health Coordinator\r\nwill work collaboratively with sites they are responsible for and local\r\npartners and will report to the MMT Project Manager. The essential\r\nresponsibilities include:\r\n- Coordinate the obtaining and/or development/ adaptation of MMT related\r\nguides and prot...",2004-02-22,Health Coordinators (two positions are open),World Vision Armenia
3,"World Vision Armenia\r\nTITLE: Health Coordinator\r\nTERM: Full-time\r\nSTART DATE/ TIME: This position starts in May 2004\r\nLOCATION: World Vision Armenia' National office, Yerevan with extensive\r\ncountrywide travel. Candidate must be flexible team player willing to\r\ntravel extensively to field locations\r\nJOB DESCRIPTION: WORLD VISION ARMENIA announces full-time position of\r\nHealth Coordinator for the implementation of a Mobile Medical Teams and\r\nPrimary Health care project.\r\nMMT Health Coordinator will be responsible for direct coordination,\r\nsupervision and technical monitoring of the program success and\r\nconstrains in Lori.\r\nJOB RESPONSIBILITIES:\r\n- Coordinate the obtaining and/or development/adaptation of MMT related\r\nguides and protocols during the start-up pha

In [82]:
cos_count = { "-2":  [0] * 12, "-1":  [0] * 12, "0":  [0] * 12, "1":  [0] * 12}
cos_count = pd.DataFrame(cos_count)
cos_count.index = range(2004, 2016, 1)
cos_count # we are making a dataframe to be able to display and plot our results

,-2,-1,0,1
2004,0,0,0,0
2005,0,0,0,0
2006,0,0,0,0
2007,0,0,0,0
2008,0,0,0,0
2009,0,0,0,0
2010,0,0,0,0
2011,0,0,0,0
2012,0,0,0,0
2013,0,0,0,0


In [84]:
for year in range(2004, 2016): # clustering each year
    year_df = remote_jobs[(remote_jobs['date'].dt.year == year)]
    for level in range(-2, 2): # getting patents in range between 2 years back all the way up to one year ahead
        curr_level = df_patent[(df_patent['patent_date'].dt.year == (year + level))] # current cluster of patents, dependent on year and shift
        if len(curr_level) == 0: continue
        for row_index, row in year_df.iterrows():
            curr = year_df.loc[row_index]
            for patent_row in range(curr_level.index[0], curr_level.index[-1], 50):
                doc1 = curr['jobpost']
                doc2 = curr_level['patent_abstract'].loc[patent_row]
                docs = [doc1, doc2]
                tfidf_vector = vectorizer.fit_transform(docs)
                similarities = cosine_similarity(tfidf_vector)
                if similarities[0][1] > 0.5:
                    cos_count.loc[year, str(level)] += 1
        

      -2  -1  0  1
2004   4  10  4  1
2005   0   0  0  0
2006   0   0  0  0
2007   0   0  0  0
2008   0   0  0  0
2009   0   0  0  0
2010   0   0  0  0
2011   0   0  0  0
2012   0   0  0  0
2013   0   0  0  0
2014   0   0  0  0
2015   0   0  0  0
      -2  -1  0  1
2004   4  10  4  1
2005   5   7  1  1
2006   0   0  0  0
2007   0   0  0  0
2008   0   0  0  0
2009   0   0  0  0
2010   0   0  0  0
2011   0   0  0  0
2012   0   0  0  0
2013   0   0  0  0
2014   0   0  0  0
2015   0   0  0  0
      -2  -1   0   1
2004   4  10   4   1
2005   5   7   1   1
2006  96  87  84  72
2007   0   0   0   0
2008   0   0   0   0
2009   0   0   0   0
2010   0   0   0   0
2011   0   0   0   0
2012   0   0   0   0
2013   0   0   0   0
2014   0   0   0   0
2015   0   0   0   0
      -2  -1   0   1
2004   4  10   4   1
2005   5   7   1   1
2006  96  87  84  72
2007  58  59  75  73
2008   0   0   0   0
2009   0   0   0   0
2010   0   0   0   0
2011   0   0   0   0
2012   0   0   0   0
2013   0   0   0   0
20